In [11]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

In [12]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [13]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [14]:
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/'+str(year)+'_cutflow.pkl'
if os.path.isfile(picklename):
    repickle=False

In [15]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['cutflow'].scale(scale_lumi, 'dataset')
    cutflow = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(cutflow, outfile, protocol=-1)
    outfile.close()

Missing file 2416 dict_keys(['WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8'])


/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:376: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7ff67161a460>
  warnings.warn(


In [16]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))

In [17]:
cutflow = templates.sum('genflavor').integrate('region','signal')
cutflow

<Hist (process,cut) instance at 0x7ff614422460>

In [18]:
cutflow.values()

{('ZH',): array([364.87708902, 364.87708902, 364.87708902,  94.04675802,
         94.04675802,  89.56093767,  54.63613886,  25.96620918,
         19.50533878,  16.5283468 ,  14.30561118,   5.55088631,
          0.        ,   0.        ,   0.        ]),
 ('WH',): array([325.97119887, 325.97119887, 325.97119887,  90.44894466,
         90.44894466,  85.73177902,  49.51309033,  28.10041756,
         17.63157394,  14.72563217,  11.44041635,   3.50978478,
          0.        ,   0.        ,   0.        ]),
 ('ttH',): array([1156.72913284, 1156.72913284, 1156.72913284,  433.92247017,
         433.92247017,  365.44726208,  128.19273077,   43.01119695,
          32.94007601,   21.79819751,   21.70239789,    5.73176095,
           0.        ,    0.        ,    0.        ]),
 ('VBF',): array([752.02557417, 752.02557417, 752.02557417,  89.05491792,
         89.05491792,  86.40647151,  47.77517252,  39.32086374,
         37.31352414,  35.95350003,  32.78736292,  21.36195267,
          0.        ,  

In [19]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]

df1 = df1[3:-3].astype('int')
df1.index = ['Jet kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','2 AK4 jets','DeepDoubleB']

df1.to_latex(buf=year+'/cutflow-sig.tex')

In [20]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[3:-3].astype('int')
df2.index = ['Jet kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','2 AK4 jets','DeepDoubleB']

df2.to_latex(buf=year+'/cutflow-bkg.tex')